# The Julia programming language

### A look at arrays and Trilinos integration

<br/>
### FOSDEM 2018
<br/>
Bart Janssens

# Outline
* A bit about Julia
<img src="images/julia-logo.svg" width=200/>
* Then something about integration
<table>
<tr>
    <td><img src="images/logo_trilinos_moon.png"/></td>
    <td><img src="images/mpi-forum-logo.jpg"/></td>
</tr>
</table>
 


# About Julia
## Why another language?
* Solve the "two language problem":
  - Prototype in a simple language
  - Write production code in a fast language

## So what is it?
* High-level programming language for scientific computing
* Dynamic language
* Strongly typed, with user-defined types and generics
* JIT-compiled using LLVM
* Native interface to C
* Central concept: **multiple dispatch**

## A simple function

In [1]:
function add(a,b)
    return a + b
end

add (generic function with 1 method)

Shorter version:

In [2]:
add(a,b) = a+b

add (generic function with 1 method)

In [3]:
add(1,2)

3

In [4]:
add(1.0, 2.0)

3.0

## Where are the types?

In [5]:
typeof(1)

Int64

In [6]:
typeof(add(1,2))

Int64

In [7]:
typeof(add(1.0,2))

Float64

## A look at the assembly
Each combination of types for the arguments compiles a different version of the code:

In [8]:
@code_native add(1,2)

	.text
Filename: In[2]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	leaq	(%rdi,%rsi), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


In [9]:
@code_native add(1.0,2.0)

	.text
Filename: In[2]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	vaddsd	%xmm1, %xmm0, %xmm0
	popq	%rbp
	retq
	nopw	(%rax,%rax)


## Equivalence with C++
The Julia function `add(a,b) = a+b` is equivalent to the following C++ function:

```c++
template<typename A, typename B>
auto add(A a, B b) -> decltype(a + b)
{
    return a + b;
}
```

* Template function valid for any type `A` and `B`
* C++ compiles a new version for every combination of types
* Automatic (static) computation of the return type (`decltype` annotation)

## Multiple dispatch
* In Julia, not writing types means any type can be used
* It still tracks the types and computes the correct return type
* Each combination of argument types yields a newly compiled version of the function
* Type computation can happen dynamically (at runtime) or statically (during JIT compilation)
* The static type computation is equivalent to the C++ version
* Deciding which function to call is **multiple dispatch** (static or dynamic)

## Types

In [10]:
# Define a type
struct MyNumber
    n::Int
end

In [11]:
# Create an instance
const mynum = MyNumber(2)

MyNumber(2)

## What about our `add` function?

In [12]:
add(mynum, 2)

LoadError: [91mMethodError: no method matching +(::MyNumber, ::Int64)[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  +([91m::Complex{Bool}[39m, ::Real) at complex.jl:247
  +([91m::Char[39m, ::Integer) at char.jl:40
  ...[39m

In [13]:
add(a::MyNumber, b) = MyNumber(a.n + b)

add (generic function with 2 methods)

In [14]:
methods(add)

# 2 methods for generic function "add":
add(a::MyNumber, b) in Main at In[13]:1
add(a, b) in Main at In[2]:1

In [15]:
add(mynum, 2)

MyNumber(4)

# Interoperability
* Built-in: C
* Through packages:
  - C++: [Cxx.jl](https://github.com/Keno/Cxx.jl) and [CxxWrap.jl](https://github.com/JuliaInterop/CxxWrap.jl)
  - Python: [PyCall.jl](https://github.com/JuliaPy/PyCall.jl)
  - R, Matlab, ...

## Calling C functions
Using `ccall`:

In [16]:
ccall(:fabs, Float64, (Float64,),-1)

1.0

The overhead is low:

In [17]:
using BenchmarkTools

In [18]:
@btime ccall(:fabs, Float64, (Float64,),-1.0)

  2.832 ns (0 allocations: 0 bytes)


1.0

In [19]:
@btime abs(-1.0)

  1.808 ns (0 allocations: 0 bytes)


1.0

## MPI
* The [MPI.jl](https://github.com/JuliaParallel/MPI.jl) package uses `ccall` to provide access to MPI from within Julia.
* Code is executed using `mpirun julia myscript.jl`
* Next, we will compare between C and Julia using a simple array sum using `MPI_Reduce`

### C

```c
#include <stdlib.h>
#include <stdio.h>
#include <sys/time.h>
#include <mpi.h>

int main(int argc, char** argv)
{
    int rank, size;
    double* array = NULL;
    const int N = 1024*1024*100;
    double mysum[1];
    double globalsum[1];
    struct timeval  t1, t2;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    const int myN = N / size;

    array = (double*)malloc(sizeof(double)*myN);
    for(int i = 0; i != myN; ++i)
        array[i] = rank+1;

    gettimeofday(&t1, NULL);

    mysum[0] = 0.0;
    for(int i = 0; i != myN; ++i)
        mysum[0] += array[i];

    MPI_Reduce(mysum, globalsum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);

    gettimeofday(&t2, NULL);
    double time_spent = (double) (t2.tv_usec - t1.tv_usec) / 1000000 + (double) (t2.tv_sec - t1.tv_sec);

    double expected_result = 0;
    for(int i = 0; i != size; ++i)
        expected_result += (i+1)*myN;

    if(rank == 0) {
        if(expected_result != globalsum[0])
            printf("Wrong result, expected %f, got %f\n", expected_result, globalsum[0]);
        printf("C Sum time: %f s\n", time_spent);
    }

    free(array);
    MPI_Finalize();
    return 0;
}

```

### Julia

In [20]:
using MPI
using Base.Test

const  N = 1024*1024*100

MPI.Init()
rank = MPI.Comm_rank(MPI.COMM_WORLD)
size = MPI.Comm_size(MPI.COMM_WORLD)

const  myN = N ÷ size # Integer division
const array = fill(rank+1.0,myN)

function mpi_sum(a)
    mysum = 0.0
    for x in a
        mysum += x
    end
    return MPI.Reduce(mysum, +, 0, MPI.COMM_WORLD)
end

expected = sum(myN*(1:size))
for i in 1:3
    rank == 0 ? (@time @test expected == mpi_sum(array)) : mpi_sum(array)
end

MPI.Finalize()

  0.124784 seconds (3.26 k allocations: 186.788 KiB)
  0.101493 seconds (13 allocations: 704 bytes)
  0.097252 seconds (13 allocations: 704 bytes)


### Timings on 4 cores

#### C
0.022 s - 0.065 s

#### Julia
0.024 s - 0.027 s

Both are equivalent, but the Julia code is a lot simpler


## CxxWrap.jl
* Uses `ccall` to access pointers to C++ functions using C
* Interfacing code is written in C++
* Julia functions generated automatically
* Approach similar to Boost.Python

#### Some timings
Loop over 50 M elements doing `double` multiplication:
```
Julia:
  0.066912 seconds (4 allocations: 160 bytes)
ccall in Julia loop:
  0.104013 seconds (4 allocations: 160 bytes)
CxxWrap.jl call in Julia loop:
  0.106241 seconds (4 allocations: 160 bytes)
C++, loop in the C++ code:
  0.064130 seconds (4 allocations: 160 bytes)
Julia cfunction callback in C++ loop
  0.277433 seconds (4 allocations: 160 bytes)
```

## [Trilinos.jl](https://github.com/barche/Trilinos.jl)

* Interface to some parts of the [Trilinos](https://trilinos.org/) C++ library
* Focus on the Tpetra matrix library and iterative solvers
* Uses CxxWrap.jl
* Example: 2D Laplace in C++ and Julia

### Laplace example:
Solve the equation $\nabla^2 \varphi + f = 0 $ with $f = 2h^2((1-x^2)+(1-y^2))$ and 0 on the boundary:

<img src="images/laplace2d.png" width=700px>

### C++ code
```c++
template<typename MatrixT>
void fill_laplace2d(MatrixT& A, const CartesianGrid& g)
{
  Teuchos::TimeMonitor local_timer(*fill_time);
  const auto& rowmap = *A.getRowMap();
  const local_t n_my_elms = rowmap.getNodeNumElements();

  // storage for the per-row values
  global_t row_indices[5] = {0,0,0,0,0};
  scalar_t row_values[5] = {4.0,-1.0,-1.0,-1.0,-1.0};

  for(local_t i = 0; i != n_my_elms; ++i)
  {
    const global_t global_row = rowmap.getGlobalElement(i);
    const local_t row_n_elems = laplace2d_indices(row_indices, global_row, g);
    row_values[0] = 4.0 - (5-row_n_elems);
    A.replaceGlobalValues(global_row, Teuchos::ArrayView<global_t>(row_indices,row_n_elems), Teuchos::ArrayView<scalar_t>(row_values,row_n_elems));
  }
}
```

### Julia code
```julia
function fill_laplace2d!(A, g::CartesianGrid)
  rowmap = Tpetra.getRowMap(A)
  n_my_elms = Tpetra.getNodeNumElements(rowmap)

  # storage for the per-row values
  row_indices = [0,0,0,0,0]
  row_values = [4.0,-1.0,-1.0,-1.0,-1.0]

  for i in 0:n_my_elms-1
    global_row = Tpetra.getGlobalElement(rowmap,i)
    row_n_elems = laplace2d_indices!(row_indices, global_row, g)
    row_values[1] = 4.0 - (5-row_n_elems)
    Tpetra.replaceGlobalValues(A, global_row, Teuchos.ArrayView(row_indices,row_n_elems), Teuchos.ArrayView(row_values,row_n_elems))
  end
end
```

### Timing comparison for 1000 x 1000 matrix (in ms)
|                  |C++  |Julia|
|------------------|-----|-----|
|Graph construction|190.7|115.2|
|Source term       |41.17|32.01|
|Matrix filling    |137.1|102.5|
|Dirichlet setup   |0.899|0.761|
|Check time        |41.06|29.42|

# Conclusions

* Julia is a fast high-level language
* It offers excellent interoperability to reuse existing libraries
* It delivers on the promise of matching C/C++ performance

## There is much more...
* Generic types and functions
* Julia parallel programming
* Metaprogramming: Julia expression trees
* Macros
* Generated functions (for e.g. loop-unrolling)
* [CUDAnative.jl](https://github.com/JuliaGPU/CUDAnative.jl): Write CUDA kernels in Julia
* Many more excellent packages

# Detailed examples

## Some dispatching benchmarks


In [21]:
using BenchmarkTools

In [22]:
@btime add(1,2)

  1.554 ns (0 allocations: 0 bytes)


3

In [23]:
# Test function with loop
function sum_arr1(array)
    result = 0
    for x in array
        result += x
    end
    return result
end

sum_arr1 (generic function with 1 method)

### The test arrays:

In [24]:
const arr1 = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [25]:
const arr2 = [1.0, 2.0, 3.0]

3-element Array{Float64,1}:
 1.0
 2.0
 3.0

In [26]:
const arr3 = Any[1, 2.0, 3]

3-element Array{Any,1}:
 1  
 2.0
 3  

### The results:

In [27]:
@btime sum_arr1(arr1)

  4.621 ns (0 allocations: 0 bytes)


6

In [28]:
@btime sum_arr1(arr2)

  67.244 ns (8 allocations: 128 bytes)


6.0

In [29]:
@btime sum_arr1(arr3)

  80.644 ns (2 allocations: 32 bytes)


6.0

### Fixing the problem

In [30]:
function sum_arr2(array)
    result = zero(eltype(array)) # 0 here was only valid for Int
    for x in array
        result += x
    end
    return result
end

sum_arr2 (generic function with 1 method)

In [31]:
@btime sum_arr2(arr1)

  4.114 ns (0 allocations: 0 bytes)


6

In [32]:
@btime sum_arr2(arr2)

  4.621 ns (0 allocations: 0 bytes)


6.0

In [33]:
# Won't work:
#@btime sum_arr2(arr3)

## Implementing a Julia array over a Kokkos view from Trilinos

The Julia type:
```julia
"""
Expose a view using its raw data pointer
"""
struct PtrView{ScalarT,N,LayoutT} <: AbstractArray{ScalarT,N}
    ptr::Ptr{ScalarT}
    size::NTuple{N,Int}
    stored_view # prevent GC of the actual view
end

```

## Implementing a Julia array over a Kokkos view from Trilinos

The functions to implement:

```julia
# Zero-based indexing
using CustomUnitRanges: filename_for_zerorange
include(filename_for_zerorange)

# Indexing directives
Base.IndexStyle{ScalarT,N,LayoutT}(::Type{PtrView{ScalarT,N,LayoutT}}) = IndexLinear()
Base.indices{ScalarT,N}(v::PtrView{ScalarT,N,LayoutLeft}) = map(ZeroRange, v.size)
Base.indices{ScalarT,N}(v::PtrView{ScalarT,N,LayoutLeft},d) = ZeroRange(v.size[d])

# Operator []
Base.getindex{ScalarT}(v::PtrView{ScalarT,1,LayoutLeft}, i::Integer) = unsafe_load(v.ptr,i+1)
Base.setindex!{ScalarT}(v::PtrView{ScalarT,1,LayoutLeft}, value, i::Integer) = unsafe_store!(v.ptr,value,i+1)
Base.getindex{ScalarT,N}(v::PtrView{ScalarT,N,LayoutLeft}, i::Integer) = unsafe_load(v.ptr,i)
Base.setindex!{ScalarT,N}(v::PtrView{ScalarT,N,LayoutLeft}, value, i::Integer) = unsafe_store!(v.ptr,value,i)

# Construction of array with the same shape
Base.similar{ScalarT,N,LayoutT}(v::PtrView{ScalarT,N,LayoutT}) = Array{ScalarT,N}(length.(indices(v)))
```